In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import numpy as np
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import folium.features
import folium.map
import folium
from folium import Circle
from PIL import Image
import os
import geopy.distance
import re

## Custom Functions

In [4]:
def screenshot(location, output_file, zoom=20, driver=None, last_request_time=None, i=0):

    #check if request has been made recently
    if last_request_time is not None:
        current_time = time.time()
        elapsed = current_time - last_request_time
        if elapsed < 10:  
            delay = np.random.uniform(10, 15) - elapsed  # add more random time
            time.sleep(delay)

    try:
        # Setup Chrome options
        if driver is None:
            chrome_options = webdriver.ChromeOptions()
            chrome_options.add_argument('--headless')
            chrome_options.add_argument('--window-size=1920,1080')
            chrome_options.add_argument('--disable-gpu')
            driver = webdriver.Chrome(options=chrome_options)


        lat, lon = location
        url = f"https://www.google.com/maps/@{lat},{lon},{zoom}z/data=!3m1!1e3!5m1!1e4!4m2!1m1!1s+!5m1!1e1"

        driver.get(url)

        # Wait for map to load
        wait = WebDriverWait(driver, 10)
        actions = ActionChains(driver)
        wait.until(EC.presence_of_element_located((By.ID, "scene")))
        
        #find layers button, click if i=0 for satellite imagery
        layers_button = wait.until(EC.element_to_be_clickable((
        By.XPATH, "//button[@aria-labelledby='widget-minimap-icon-overlay']")))
        if i == 0:
            layers_button.click()
        #hover over layers button to reveal more button
        actions.move_to_element(layers_button).perform()
        time.sleep(10)

        #find and click more button
        more_button = wait.until(EC.element_to_be_clickable((
            By.XPATH, "//button[@jsaction='layerswitcher.quick.more']//label[text()='More']/..")))
        more_button.click()
        time.sleep(10)

        #find and turn off labels
        labels_button = wait.until(EC.presence_of_element_located((By.XPATH,
                                                                    "//div[@class='yYTQHb']/ul[@class='XOaW9c']/li[@class='opFIdd'][2]/button[@jsaction='layerswitcher.intent.labels']")))
    
        labels_button.click()

        # find and click exit menu button
        x_button = wait.until(EC.presence_of_element_located((By.XPATH,
                                                                    "//div[@class='OwXc3d']/header[@class='sjBOQb']/button[@jsaction='layerswitcher.close']")))
        
        x_button.click()
        
        
        # build in more time to prevent errors
        time.sleep(10)
        
        # screenshot
        driver.save_screenshot(output_file)
        
        # driver.quit()
        return True, driver, time.time()
        
    except TimeoutException:
        print("Timeout waiting for Google Maps to load")
        print(f"i = {i}")
        if driver:
            driver.quit()
        return False, None, time.time()
    except Exception as e:
        print(f"Error capturing screenshot: {str(e)}")
        print(f"i = {i}")
        if driver:
            driver.quit()
        return False, None, time.time()

def batchScreenshots(locations, output_template):
    driver = None
    last_request_time = None
    
    try:
        for i, location in enumerate(locations):
            output_file = output_template.format(i)
            success, new_driver, timestamp = screenshot(
                location, 
                output_file, 
                driver=driver,
                last_request_time=last_request_time,
                i=i #return i for satellite clicking
            )
            
            if success:
                driver = new_driver
                last_request_time = timestamp
            else:
                # if failed, reset driver and continue, this messes up satellite
                driver = None
                time.sleep(60)  # more delay after failure
                
    finally:
        if driver:
            driver.quit()


In [5]:
def mapPoints(dataframe):
    #initialize map
    map = folium.Map(location=(39.75863161320975, -105.02562868409278), zoom_start=12, 
                          max_zoom=20,
                          tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
                          attr='Google')

    #loop through dataframe rows, extract location and crash count
    for i, row in dataframe.iterrows():
        tempList = [row["geo_lat"], row["geo_lon"]]
        
        crashCount = row["crash_count"]

        #color code based on crash count
        if crashCount == 0:
            color = 'green'
        elif crashCount <= 25:
            color = 'blue'
        elif crashCount <= 50:
            color = 'blue'
        elif crashCount <= 75:
            color = 'orange'
        elif crashCount <= 100:
            color = 'red'
        else:
            color ='darkred'
  
        radius = 30.48 # 100ft in meters
        id = str(i)
        # Add a circle with 100ft radius for visualization
        map.add_child(Circle(location=tempList, radius=radius,color=color, fill=False))
        map.add_child(folium.Marker(location=tempList, popup=f"{id}, crashes: {crashCount}",
                                         icon=folium.Icon(color=color)))
        
    #save map
    map.save("Denver_intersections.html")

In [14]:
def cropImage(imageFile, outputFile, top=100, bottom= 90, final = 1574):
    image = Image.open(imageFile)

    #margins to cut (pixels)
    top_margin = top
    bottom_margin = bottom
   
    #image size for cropping 
    final_size = final
    side_crop = (image.width - final_size) // 2 

    # crop image
    cropped = image.crop((
    side_crop,          # left
    top_margin,         # top
    image.width - side_crop,  # right
    image.height - bottom_margin  # bottom
    ))

    # print(cropped.size) 
    cropped.save(outputFile)
    image.close()

In [7]:
def isNearby(coord1, coord2, threshold=100):
    #get distance, return True and distance if within threshold distance
    dist = geopy.distance.distance(coord1, coord2).ft
    if dist <= threshold:
        return True, dist
    else:
        return False, None


## Capture Screenshots

### Intersections Lists

In [9]:
completedIntersections = [
    (39.78020645317515, -105.05327226516985), (39.77838637879373, -105.05324704869999), (39.77660123174248, -105.0532692374875), 
    (39.774769742332175, -105.05325584260476), (39.782811171764415, -105.00642805916023), (39.75965564500341, -105.01182468251663), 
    (39.7596898761914, -105.01107081349411), (39.75923625573741, -105.01052900731176), (39.76018782779032, -105.00930703566384), 
    (39.761129969826925, -105.00809455106103), (39.762023789847326, -105.00703175202467), (39.781868668713024, -105.0532420772096), 
    (39.782827386530165, -105.05325924800752), (39.78386835377152, -105.05325499796076), (39.78452198443768, -105.05327112369883), 
    (39.78574796425909, -105.05326156861274), (39.79106491964947, -105.05324538359004), (39.77295676049358, -105.0532680018107), 
    (39.77112971245031, -105.05330447942929), (39.76934467239783, -105.05326192220797), (39.7657281121043, -105.05327220005115),
    (39.76390966169606, -105.05325424500843), (39.76209172041639, -105.05326032461129), (39.760297098883804, -105.0532481654053),
    (39.758465041223836, -105.0532238469931), (39.75848994347928, -105.0487043247645), (39.758504782718234, -105.0440089252607),
    (39.75851432773611, -105.03932416970807), (39.75851381426044, -105.0346248338673), (39.758482215968186, -105.02992011414406),
    (39.75849646585114, -105.02526887074178), (39.758511245875106, -105.02163332825067), (39.75844817984832, -105.01585143655939),
    (39.758399374191065, -105.011758117638), (39.76322528290787, -105.00643916142329), (39.76444059157726, -105.00645753873509),
    (39.76565076381565, -105.00645344467183), (39.76685628567768, -105.0064577772172), (39.76738517754211, -105.00645901439394),
    (39.768062059800094, -105.00645699924749), (39.769293260912754, -105.006467855351), (39.76928932710426, -105.005366726852),
    (39.77053170212778, -105.00645471719922), (39.771739201277455, -105.00645229422388),
    (39.77298698085316, -105.00645315485723), (39.77421689964544, -105.00645527546958), (39.77544756340408, -105.00645002498462),
    (39.776542029821016, -105.00644595596819), (39.77903626822966, -105.00645232136387), (39.78016017980555, -105.00645554770544),
    (39.78203814372065, -105.00644710538911), (39.78375334386847, -105.00645111726796), (39.78545862443461, -105.00643586571303),
    (39.786270858607715, -105.00643215100672), (39.786577118632444, -105.00814048764784), (39.78706866189822, -105.01018659500093), 
    (39.787453777491116, -105.01023818145433), (39.787451077344976, -105.01121018394467), (39.78743710227759, -105.01238986271177), 
    (39.78743070809555, -105.01353461879044), (39.787431924870305, -105.01471663760671), (39.78744521790409, -105.01585682440671), 
    (39.78743692797986, -105.01700110162282), (39.78743830142271, -105.01816538273279), (39.787436242993216, -105.01932852836758),
    (39.78743604604747, -105.02048375803656), (39.78743871639314, -105.02165167993468), (39.78744075712096, -105.02280984340909),
    (39.78743293503274, -105.02396234282013), (39.78745705273132, -105.02521877535591), (39.78747724451223, -105.0275884218127),
    (39.78748528517835, -105.02874223915474), (39.78748358258507, -105.02990667619153), (39.787493990015484, -105.03143254957101),
    (39.78747953027653, -105.0322170863441), (39.787449895432815, -105.03460911735992), (39.78744197848432, -105.0357919137042),
    (39.78744474245819, -105.03695900949778), (39.7874370056027, -105.03810706162876), (39.78743656265784, -105.03927551704979),
    (39.78743586613522, -105.0404271964994), (39.78744098847525, -105.04159925791002), (39.78743831304456, -105.04275391193295),
    (39.78744133057393, -105.04396324915784), (39.78562443822328, -105.04395274561756), (39.783878790655805, -105.04395904809117),
    (39.7831885365965, -105.04396263060232), (39.78207584308399, -105.04396763270348), (39.78022483885559, -105.04399078765366),
    (39.77838118772822, -105.04398260105856), (39.776585847863565, -105.04398802418466), (39.77529554090092, -105.04400177733592),
    (39.77476263509053, -105.0440127915723), (39.77294061634813, -105.04404351734996), (39.77249146198811, -105.04403025935277),
    (39.77104968752806, -105.04403517822348), (39.76937202212908, -105.0440362337737), (39.76756468850071, -105.04402977737136),
    (39.76665560133304, -105.04403735825862), (39.7657496117969, -105.04403144037843), (39.76482580605524, -105.04402598997547),
    (39.76392884871359, -105.04401746820581), (39.763015898305746, -105.04402811106779), (39.76212090395296, -105.04402041166865),
    (39.76120823644629, -105.04401690295327),(39.76031288039841, -105.04401050051577), (39.78562112173635, -105.03928409981931),
    (39.783264037799036, -105.03927198282202), (39.78208750274448, -105.0392759710875), (39.77838590161048, -105.0392699436596),
    (39.77567858443067, -105.039306787309), (39.77526954416455, -105.03932865306949), (39.77386664736948, -105.03932663038535),
    (39.773079251343695, -105.03932735638234), (39.77246377176957, -105.03934189839572), (39.77126577489896, -105.03934322165883),
    (39.76944196287453, -105.03935923709759), (39.76762893640711, -105.03934374837323),(39.76578901107697, -105.03933375995574),
    (39.7639669263521, -105.03933475481445), (39.76212746759918, -105.03932245335118), (39.760325452378495, -105.03932692433835),
    (39.759444542573455, -105.0393262933376), (39.78562353414779, -105.0345883516716), (39.78303572889719, -105.03461358605176), 
    (39.782119654710264, -105.03461286842658), (39.781084788468775, -105.03461553918642), (39.7791409811353, -105.03462905280976),
    (39.77838920452578, -105.03462528219525), (39.77796562405021, -105.03462700584419), (39.77498538811972, -105.03462027377779),
    (39.77316143848167, -105.03462683577966), (39.77196137540048, -105.03462957526385), (39.770762187579784, -105.03464373548238),
    (39.76956325274809, -105.03464420924014), (39.76832551232982, -105.03465776623786), (39.7676793235245, -105.03466038136493),
    (39.76648481993121, -105.03464515967157), (39.765760876847345, -105.03464701999158), (39.76497104698917, -105.03463701835378),
    (39.7639515401367, -105.03463589037753), (39.76299524287288, -105.03462315808595), (39.76214228473493, -105.03463221072994),
    (39.76123485614694, -105.03462078955155), (39.76032429520983, -105.03462833204709), (39.75944169797527, -105.0346303858852),
    (39.78563185709938, -105.0299063646194), (39.78025750742648, -105.02991967725099), (39.779099241539946, -105.02991505515477),
    (39.777907737677914, -105.0299231060866), (39.77485376670588, -105.02995021083726), (39.77303285104486, -105.02996302484348),
    (39.77212450228195, -105.02996578731351), (39.77182496440487, -105.02998343798289), (39.77122727947795, -105.02997015922236),
    (39.770624587616425, -105.02998797144603), (39.77033383145626, -105.02996632450251), (39.7694165165604, -105.02997961161141),
    (39.76833920496926, -105.02995219481089), (39.76740888097393, -105.02995106430201), (39.7664958115997, -105.02995096147406), 
    (39.76562760945352, -105.02994595418188), (39.764713357673735, -105.03000391521566), (39.76389715528169, -105.03005564120846), 
    (39.762130473174444, -105.0300032918647), (39.761228310972186, -105.02993643613415), (39.76085984718627, -105.02992889588548), 
    (39.760311699723815, -105.029936483496), (39.75942256461298, -105.02993648712551), (39.76098914673373, -105.02679918240682), 
    (39.78561448733205, -105.02521423570458), (39.783910657259575, -105.02521604465153), (39.78215898426814, -105.02521653358133),
    (39.77904068800478, -105.0252290080337), (39.7784507279766, -105.02522426134985), (39.777855587321696, -105.02523737630197),
    (39.77532969253662, -105.02523739881549), (39.77472033735847, -105.02525699776427), (39.7741262254356, -105.0252395810507),
    (39.77291394400391, -105.02525744654555), (39.7720287724949, -105.02526908552619), (39.771102720212234, -105.02525796350031),
    (39.77021790816687, -105.0252718566838), (39.769284389472524, -105.0252763688273), (39.76836932866761, -105.02527880675322),
    (39.76803758664244, -105.02524756851683), (39.76683827974178, -105.02524890019208), (39.76650257311227, -105.025249971051),
    (39.765603477199015, -105.02524753439226), (39.76466269463694, -105.02524685635834),
    (39.76325892176075, -105.0252449760129), (39.7620103056824, -105.02524970624947), (39.7602417642918, -105.02522282914947),
    (39.78562352382935, -105.01585857859722), (39.78328474781535, -105.0158450139986), (39.782559592462285, -105.01584760749466),
    (39.78199250501296, -105.01584105987874), (39.78135640489605, -105.01584606225008), (39.78020444113949, -105.01585352072853),
    (39.778974127241, -105.01586322925786), (39.77777164999193, -105.0158698489505), (39.77534637960975, -105.01589134299343),
    (39.77413017076778, -105.01590027948536), (39.772931392088914, -105.01589374356854), (39.77172700403608, -105.01589683432297),
    (39.770521616563165, -105.01588238940373), (39.76928711062639, -105.01588637933341), (39.76806454826839, -105.01587168556215),
    (39.76685629700473, -105.01587540157458), (39.7656432301861, -105.015881167812), (39.76443462642701, -105.01587013212834),
    (39.76322387941316, -105.01586869968705), (39.762024797001594, -105.01585394753714), (39.760869249097425, -105.01585496277892), 
    (39.760204585824525, -105.01585591181292), (39.758958289475686, -105.01588032883348), (39.78554301493439, -105.0112125371891), 
    (39.783255246910066, -105.01115129760696), (39.78256500297324, -105.01115375094219), (39.781359337891644, -105.01115647345017),
    (39.77940094567678, -105.01114748873825), (39.77896816022616, -105.01115092003946), (39.77775537409778, -105.01116711769856),
    (39.77534761177024, -105.0112375979903), (39.77413474557024, -105.01123601341504), (39.77293206867928, -105.0112365963939),
    (39.771730593686755, -105.01123052815281), (39.770514330882406, -105.01123254681993), (39.76928974271347, -105.01121602437891),
    (39.76805772488123, -105.01121439958453), (39.76685934630196, -105.01121901561038), (39.76565346403115, -105.01121000408335),
    (39.76443276809249, -105.01120519579965), (39.7632290559434, -105.01119641156674), (39.76203077802503, -105.01119179065083),
    (39.760799380671884, -105.01115500026319), (39.783855566149185, -105.00825321959015), (39.7838534243284, -105.0092227246181),
    (39.78383688386098, -105.01019728921813), (39.783850009575886, -105.01118989316804), (39.78383241928006, -105.01237908373996),
    (39.78384702349633, -105.01354050556591), (39.78392711748531, -105.01471480691464), (39.783989825208494, -105.01585397122868),
    (39.78394066150258, -105.0170040949186), (39.783912231064924, -105.01817208662486), (39.78391319665649, -105.01933383739359),
    (39.78396265693193, -105.02049151946122), (39.784209702259574, -105.02167058472034), (39.78449074045283, -105.02283290665406),
    (39.78413553634888, -105.02989404077395), (39.78391063925807, -105.03104565865517), (39.78385639443114, -105.03223214257494), 
    (39.78383835578301, -105.03339280597326), (39.78375509125905, -105.03459501352208), (39.78388670225713, -105.03578836172241), 
    (39.78388878980521, -105.03695697197541), (39.78388668570997, -105.0381189998416), (39.78388755512739, -105.03926949237945),
    (39.78388287969295, -105.04043473557434),  (39.78388763376824, -105.04158990223338), (39.78388534496175, -105.04275776569979), 
    (39.78023833678181, -105.04514186003449), (39.780228307246574, -105.04629467954162), (39.78023515946065, -105.04746245082778),
    (39.78023399869305, -105.04861401573791), (39.78023427338514, -105.0497854514875), (39.78022090434584, -105.0509316481698),
    (39.780217252649585, -105.05208888283903), (39.78022072621648, -105.0416107172768), (39.78021165244498, -105.0404400319501),
    (39.780201835939536, -105.03927829864956), (39.78021179529743, -105.03811511393111), (39.78022003925565, -105.03697162878213),
    (39.7802070634338, -105.03579911765412), (39.780248701064636, -105.03462283451567), (39.780251815915854, -105.0330451282489),
    (39.78026340950785, -105.0314764958538),(39.78027301830142, -105.02835287019693), (39.78027826869944, -105.02678280714129),
    (39.78027010101169, -105.02521742653548), (39.78024821268036, -105.02396872896216), (39.78024413988471, -105.02284764725162), 
    (39.78022887413569, -105.02168360165226), (39.780225223774, -105.02052519325015), (39.78022410785952, -105.01935940061884),
    (39.78021866936991, -105.01820846827547), (39.78021011290703, -105.01706011118091), (39.78018061518372, -105.01428199293755), 
    (39.78017850855544, -105.01270328758405), (39.78017485866177, -105.01114272659008), (39.78016612756779, -105.00957462128699), 
    (39.7801658018311, -105.00802448849807), (39.77657110169734, -105.05210019603636), (39.77657654117384, -105.05093194316807),
    (39.77656754277922, -105.04977941693703), (39.77657365636336, -105.04861576646472), (39.77657032961708, -105.04746612512325),
    (39.776573719912626, -105.04629869649897), (39.77657603972191, -105.04513670107015), (39.77669021055988, -105.04280036282115),
    (39.776695663553554, -105.04160277325013), (39.77669349361697, -105.04044026518794), (39.776696955023105, -105.03929265044233),
    (39.776709932426584, -105.03811772589941), (39.77672889582203, -105.03694213339965), (39.776743852459916, -105.03578508594526),
    (39.7767564006881, -105.03462968521526), (39.776731968834554, -105.03341931264093), (39.77673571544024, -105.0330552226979), 
    (39.776719925956485, -105.03225777431665), (39.77671096767632, -105.03149343571914), (39.7767074642844, -105.0310947011189),
    (39.776710334456226, -105.02994076871502), (39.77667639591201, -105.02878719322588), (39.77668385270794, -105.0283592225894),
    (39.77666731231583, -105.02761356544048), (39.776657855068066, -105.02679735747705), (39.77666106812964, -105.0252237722639),
    (39.77663111910999, -105.0239954063945), (39.776631393778985, -105.0228472664008), (39.77662190326989, -105.02206516595803),
    (39.77663157663248, -105.02167577653589), (39.77662406392363, -105.02052107473246), (39.77660549440647, -105.0193679843073),
    (39.77660040929417, -105.01898891593537), (39.77659129198161, -105.01821521754525), (39.776575583619355, -105.01744595115339),
    (39.77658856615251, -105.01705738272085), (39.77657461061371, -105.01587394531886), (39.77655992876001, -105.01467613582311),
    (39.776565174558435, -105.0135083462787), (39.77656459768128, -105.01236543822762), (39.7765566741727, -105.01121468115919),
    (39.776539053433545, -105.01001032939253), (39.776539937177176, -105.00905005940139), (39.77654850981201, -105.00804537356474),
    (39.77652609882477, -105.00761895054951), (39.77654358623797, -105.00704648580357)] 

# 41st
forty1st = [
    (39.77294796327322, -105.052100505591), (39.77295346824296, -105.0509174538375), (39.77294862257421, -105.0497727623233),
    (39.77295147859224, -105.04861876480365), (39.772944523904386, -105.04745736840405), (39.772961747627086, -105.04630057246457),
    (39.772949358643366, -105.0451618712539), (39.77247817461568, -105.04287224444933), (39.772477023001485, -105.04167026779095),
    (39.77246912029307, -105.04047924217264), (39.773096083283, -105.03814535087226), (39.77310725013064, -105.03698204358076), 
    (39.77312644636311, -105.03581220685157), (39.77314153555342, -105.03343942074123), (39.773103678143386, -105.03227965856212),
    (39.77307571281328, -105.03111644972843), (39.77301535086137, -105.02880551313115), (39.772961120563984, -105.02760707717788),
    (39.77294963779286, -105.02363746133004), (39.77292745616744, -105.0229101797739), (39.772942003003436, -105.02207475698145),
    (39.772930731035075, -105.02053677381215), (39.77294145746159, -105.01898710466598), (39.77293220853377, -105.0174420400737),
    (39.77293016363786, -105.01469732187424), (39.772917481350646, -105.01354586110764), (39.77292547643929, -105.01240015909995),
    (39.77293422894352, -105.0099994776142), (39.77293270926276, -105.00880703941523), (39.77293578645382, -105.00763796203687)]

# 38th
thirty8th = [
    (39.76934985180375, -105.05092657306818), (39.7693658291486, -105.04977597396268), (39.769369829597835, -105.04866053334221),
    (39.7693753010757, -105.0474527089811), (39.76937461979124, -105.04629023767278), (39.76936836269833, -105.04514885563769),
    (39.769388831082495, -105.04287816283804), (39.76944603512535, -105.04169052301474), (39.76945647385256, -105.04050251796637),
    (39.7694759533899, -105.03815927433989), (39.76951371456863, -105.03700239778647), (39.76953466314681, -105.03583031461739),
    (39.76952575206974, -105.03308547521593), (39.76947544035363, -105.03231508983824), (39.769344532323785, -105.02762104980349),
    (39.76926439104738, -105.0236269303677), (39.76929179336984, -105.02290293031375), (39.769278688886914, -105.02206822973905),
    (39.76928342857522, -105.02054183632141), (39.76928156326794, -105.01897491309907), (39.76928465881759, -105.01743693613835),
    (39.769306504742396, -105.01470677624037), (39.76928005669509, -105.01280353739119), (39.76927617298384, -105.0096283912495),
    (39.76929523789575, -105.0080384559319)]

# 35th
thirty5th = [
    (39.76571050952809, -105.05262720562523), (39.765721554397246, -105.05070418331766), (39.76572853803385, -105.04878242280603),
    (39.76573459772973, -105.04685981876892), (39.765742961114086, -105.04494882297526), (39.76575483794558, -105.04285558466367),
    (39.76576632276614, -105.04167998900296), (39.76577128074972, -105.04050078991142), (39.76580410317062, -105.03814899661366),
    (39.765815301225814, -105.03698411097605), (39.765823196620026, -105.03579803785604), (39.765634091307426, -105.0322995266723),
    (39.76558188770164, -105.027602761036), (39.765630972947775, -105.02362457723925), (39.76563024218791, -105.0220777844302),
    (39.76563291892558, -105.02052181210804), (39.76563428305019, -105.01897744120747), (39.765645357235606, -105.01742422858148),
    (39.765646501228, -105.01437328873989), (39.765639905537256, -105.01279450621054), (39.765649950973625, -105.00961900051098),
    (39.76565061229379, -105.0080368511303)]

# 32nd
thirty2nd = [
    (39.762099588921686, -105.05261250767472), (39.762080516969775, -105.05208880636484), (39.762091661394784, -105.05091076477335),
    (39.76211033159299, -105.04876126228413), (39.76211754800399, -105.04686024664508), (39.76210059307185, -105.04635800644819),
    (39.76211367114831, -105.04494277976904), (39.76211667635339, -105.04283745676157), (39.762125657710925, -105.04166147505158),
    (39.762122384678285, -105.04049231296828), (39.76212322921651, -105.0381554241914), (39.76212965369212, -105.03696778106453),
    (39.762135253747545, -105.0357782596189), (39.76212952714705, -105.03227290324097), (39.762103934677356, -105.02889444619689),
    (39.76273731174553, -105.02765585314702), (39.76201652410335, -105.02361470682435), (39.762012214593895, -105.02291634469286),
    (39.76202248907064, -105.02052191341653), (39.76202845956684, -105.01897992656828), (39.76201278731597, -105.01819724474274),
    (39.76202536048245, -105.01437320946596), (39.76203414445418, -105.01279482131004), (39.76204666818601, -105.00960117273137),
    (39.762052574810006, -105.0080278490155)]

#13th
thirtieth = [
    (39.76028888907225, -105.05209494918738), (39.76029017495025, -105.0509161424077), (39.76029947412255, -105.04869657370834),
    (39.76031441458768, -105.0463516117321), (39.7603114018854, -105.0428456349069), (39.76031689358457, -105.04167910640489),
    (39.760320112461656, -105.04049936368276), (39.76032837281174, -105.03698321735271), (39.76032964957117, -105.03228557439647),
    (39.76029937710756, -105.02817754825882), (39.759649552253684, -105.01315950660685)]

complete = len(completedIntersections)
nextUp = (len(forty1st + thirty8th + thirty5th + thirty2nd + thirtieth))
print(f"{complete} + {nextUp} = {complete+nextUp}")

339 + 113 = 452


In [11]:
#check there are no intersections within 100ft of each other

#merge all intersections
allIntersections = completedIntersections + forty1st + thirty8th + thirty5th + thirty2nd + thirtieth
count = 0
points = []
for i, p1 in enumerate(allIntersections):
    for j in range(i+1, len(allIntersections)):
        p2 = allIntersections[j]
        tempDist = isNearby(p1, p2, 100)
        if tempDist is not None:
            points.append((i, j, tempDist))
            count += 1
        # if count > 2:
        #     break

#print based on closest points
# sortedpoints = sorted(points, key=lambda x: x[2])
# distList = []
# for p in sortedpoints:
#     print(p)
#     distList.append(p[2])
# print(len(sortedpoints))
# distList.sort()


### Screenshot Capture

In [ ]:
#erros kept happening when trying to capture too many points at once
output_template = "Denver_{}.png"
intersectionList = thirtieth
# print(intersectionList)
# batchScreenshots(intersectionList, output_template)
# screenshot((39.783912231064924, -105.01817208662486), "Denver_252.png")

### Crop  & Reduce Screenshots

In [16]:
#caused error, loop did not end
# for fileName in os.listdir('UncroppedScreens'):
#         newFileName = 'CroppedImages\\' + "cropped" + '_' + fileName
#         cropImage(fileName, newFileName)
        # print(newFileName)

output_dir = 'SecondCropped'
# os.makedirs(output_dir, exist_ok=True)

# for fileName in os.listdir('CroppedImages'):
#         pattern = r'CroppedImages\\'
#         updatedFile = re.sub(pattern, '', fileName)
#         fileName = 'CroppedImages\\' + fileName
#         newFileName = 'SecondCropped\\' + updatedFile
        # print(fileName)
        # print(newFileName)
        # cropImage(fileName, newFileName, 193, 190, 1000)
# print(newFileName)

# img = Image.open('SecondCropped\cropped_Denver_0.png')
# print(img.size)
# img.close()

output_template = "cropped_Denver_{}.png"
# fileName = "CroppedImages\cropped_Denver_57.png"
# pattern = r'CroppedImages\\'
# updatedFile = re.sub(pattern, '', fileName)
# newFileName = 'SecondCropped\\' + updatedFile
# print(fileName)
# print(updatedFile)
# print(newFileName)
# cropImage(fileName, newFileName)

#fix flawed crops
for i in range(6, 10):
        fileName = output_template.format(i)
        pattern = r'CroppedImages\\'
        updatedFile = re.sub(pattern, '', fileName)
        newFileName = 'SecondCropped\\' + updatedFile
        # fileName = output_template.format(i)
        # newFileName = 'CroppedImages\\' + "cropped" + '_' + fileName
        # print(fileName)
        # print(updatedFile)
        # print(newFileName)
        cropImage(fileName, newFileName)
# print(len(os.listdir('CroppedImages')))

In [ ]:
#reduce size of images for faster training
output_dir = "ReducedCropped2"
image_dir = "SecondCropped"
os.makedirs(output_dir, exist_ok=True)


target_size=(255, 255)

for img_file in os.listdir(image_dir):
# for i in range(6, 10):
    img_file = output_template.format(i)
    input_path = os.path.join(os.getcwd(), img_file)
    output_path = os.path.join(output_dir, img_file)

    with Image.open(output_path) as img:
        img = img.resize(target_size, Image.Resampling.LANCZOS)
        # print(img.size)
        img.save(output_path, 'PNG', optimize=True)



(255, 255)
(255, 255)
(255, 255)
(255, 255)


## Load & Clean Accident Data

In [ ]:
trafficData = pd.read_csv("ODC_CRIME_TRAFFICACCIDENTS5YR_P_1817589215432503270.csv")
print(trafficData.columns)
print(len(trafficData.columns))
print()

rdDesc = trafficData["ROAD_DESCRIPTION"].unique()
print(rdDesc)
print(len(rdDesc))
print()


C:\Users\tgbea\AppData\Local\Temp\ipykernel_10848\1680562906.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  trafficData = pd.read_csv("ODC_CRIME_TRAFFICACCIDENTS5YR_P_1817589215432503270.csv")


Index(['object_id', 'incident_id', 'offense_id', 'offense_code',
       'offense_code_extension', 'top_traffic_accident_offense',
       'first_occurrence_date', 'last_occurrence_date', 'reported_date',
       'incident_address', 'geo_x', 'geo_y', 'geo_lon', 'geo_lat',
       'district_id', 'precinct_id', 'neighborhood_id', 'bicycle_ind',
       'pedestrian_ind', 'HARMFUL_EVENT_SEQ_1', 'HARMFUL_EVENT_SEQ_2',
       'HARMFUL_EVENT_SEQ_3', 'road_location', 'ROAD_DESCRIPTION',
       'ROAD_CONTOUR', 'ROAD_CONDITION', 'LIGHT_CONDITION', 'TU1_VEHICLE_TYPE',
       'TU1_TRAVEL_DIRECTION', 'TU1_VEHICLE_MOVEMENT', 'TU1_DRIVER_ACTION',
       'TU1_DRIVER_HUMANCONTRIBFACTOR', 'TU1_PEDESTRIAN_ACTION',
       'TU2_VEHICLE_TYPE', 'TU2_TRAVEL_DIRECTION', 'TU2_VEHICLE_MOVEMENT',
       'TU2_DRIVER_ACTION', 'TU2_DRIVER_HUMANCONTRIBFACTOR',
       'TU2_PEDESTRIAN_ACTION', 'SERIOUSLY_INJURED', 'FATALITIES',
       'FATALITY_MODE_1', 'FATALITY_MODE_2', 'SERIOUSLY_INJURED_MODE_1',
       'SERIOUSLY_INJURE

In [8]:
#load specified columns
keptCols = ['incident_id', 'first_occurrence_date', 'geo_lon', 'geo_lat',
       'precinct_id', 'road_location', 'ROAD_DESCRIPTION']

trafficData = pd.read_csv("ODC_CRIME_TRAFFICACCIDENTS5YR_P_1817589215432503270.csv", usecols=keptCols)
# print(trafficData.info())
print("length when read in: ", len(trafficData))
#filter out all precincts but 113 and 111
keptPrecincts = [113, 111]
print("sum of NA precincts: ", trafficData['precinct_id'].isna().sum())

trafficData = trafficData[trafficData["precinct_id"].isin(keptPrecincts) | trafficData["precinct_id"].isna()]

#check
# print(trafficData["precinct_id"].unique())
# print(trafficData.info())
print("length after filtering out precincts: ", len(trafficData))

#convert occurance date to datetime - did not end up filtering on year
trafficData['first_occurrence_date'] = pd.to_datetime(trafficData['first_occurrence_date'], format="%m/%d/%Y %I:%M:%S %p")

#filter out anything before 2022 - removed this filter
# trafficData = trafficData[trafficData['first_occurrence_date'].dt.year >= 2022]
# print()
# print(trafficData.info())

#filter out any accidents not intersection related to avoid confusion with intersections
# close to highways or other roads that are blocked off from intersections
# print(trafficData["ROAD_DESCRIPTION"].unique())
roadDescriptions = ['Intersection Related', 'At Intersection', 'INTERSECTION RELATED', 
                  'AT INTERSECTION', 'ROUNDABOUT', 'Ramp Related', 'Ramp']

trafficData = trafficData[trafficData['ROAD_DESCRIPTION'].isin(roadDescriptions)]
# print()
print("length after filtering on road description: ", len(trafficData))

# print(trafficData.info())

#check for any NA values for lat and long:
print(trafficData['geo_lat'].isna().sum())
print(trafficData['geo_lon'].isna().sum())

#drop NA values, if necessary
trafficData.dropna(subset=['geo_lon'], inplace=True)
print("length after filtering out NA lat/longs: ", len(trafficData))


length when read in:  257299
sum of NA precincts:  10080
length after filtering out precincts:  25427
length after filtering on road description:  8540
2941
2941
length after filtering out NA lat/longs:  5599


In [9]:
trafficData.describe().T

,count,mean,min,25%,50%,75%,max,std
incident_id,5599.0,1801459892.164315,2014479.0,2014302808.0,2017260587.0,2020378844.0,20238054863.0,1031586004.800814
first_occurrence_date,5599,2018-05-07 21:30:16.899446272,2013-01-03 14:30:00,2015-08-08 17:55:00,2018-01-11 17:51:00,2021-01-27 04:30:00,2024-11-04 06:04:00,NaN
geo_lon,5599.0,-105.024026,-105.206197,-105.03457,-105.025189,-105.011157,-104.6,0.016257
geo_lat,5599.0,39.772457,39.665155,39.762611,39.769402,39.783265,39.839999,0.010191
precinct_id,5595.0,112.049508,111.0,111.0,113.0,113.0,113.0,0.998863


In [13]:
trafficData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5599 entries, 0 to 257191
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   incident_id            5599 non-null   int64  
 1   first_occurrence_date  5599 non-null   object 
 2   geo_lon                5599 non-null   float64
 3   geo_lat                5599 non-null   float64
 4   precinct_id            5595 non-null   float64
 5   road_location          5571 non-null   object 
 6   ROAD_DESCRIPTION       5599 non-null   object 
dtypes: float64(3), int64(1), object(3)
memory usage: 349.9+ KB


## Create Intersection Dataframe

In [ ]:
lats, longs = zip(*allIntersections)
# print(len(lats))
# print(len(longs))
intersectionDF = pd.DataFrame({'id': range(len(lats)), 'geo_lat': lats, 'geo_lon': longs})

intersectionDF['lat'] = lats
intersectionDF['long'] = longs
intersectionDF['crash_count'] = 0

intersectionDF.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           452 non-null    int64  
 1   geo_lat      452 non-null    float64
 2   geo_lon      452 non-null    float64
 3   lat          452 non-null    float64
 4   long         452 non-null    float64
 5   crash_count  452 non-null    int64  
dtypes: float64(4), int64(2)
memory usage: 21.3 KB


In [171]:
intersectionDF.head()

,id,geo_lat,geo_lon,lat,long,crash_count
0,0,39.780206,-105.053272,39.780206,-105.053272,0
1,1,39.778386,-105.053247,39.778386,-105.053247,0
2,2,39.776601,-105.053269,39.776601,-105.053269,0
3,3,39.774770,-105.053256,39.774770,-105.053256,0
4,4,39.782811,-105.006428,39.782811,-105.006428,0


### Assigning Accidents to Intersections

In [179]:
# check for accidents near intersections, add to crash count if within 100'
# if a crash is near two intersections, only add to crash count of closest one
for i, row in trafficData.iterrows():
    crashLoc = (row["geo_lat"], row["geo_lon"])
    # print(crashLoc)
    nearbyList = []
    indexList = []
    for j, intersectionRow in intersectionDF.iterrows():
        intersection = (intersectionRow["geo_lat"], intersectionRow["geo_lon"])
        nearby, dist = isNearby(crashLoc, intersection, 100)
        if nearby:
            nearbyList.append(dist)
            indexList.append(j)
    if nearbyList:
        distIndx =nearbyList.index(min(nearbyList))
        closestIndx = indexList[distIndx]
        intersectionDF.at[closestIndx, "crash_count"] += 1

intersectionDF.describe().T

,count,mean,std,min,25%,50%,75%,max
id,452.0,225.500000,130.625419,0.000000,112.750000,225.500000,338.250000,451.000000
geo_lat,452.0,39.773094,0.008505,39.758399,39.765719,39.772951,39.780218,39.791065
geo_lon,452.0,-105.028868,0.013852,-105.053304,-105.039348,-105.029933,-105.015875,-105.005367
lat,452.0,39.773094,0.008505,39.758399,39.765719,39.772951,39.780218,39.791065
long,452.0,-105.028868,0.013852,-105.053304,-105.039348,-105.029933,-105.015875,-105.005367
crash_count,452.0,7.794248,17.536902,0.000000,0.750000,2.000000,6.000000,164.000000


In [188]:
intersectionDF.to_csv('IntersectionCrashes.csv', index=False)

## Mapping Points

In [186]:
mapPoints(intersectionDF)

In [4]:
intersections = pd.read_csv('IntersectionCrashes.csv')

intersections['crash_count'].sum()

3523